In [ ]:
data BST a = Leaf |
    Node {leftChild :: BST a, val :: a, rightChild :: BST a } deriving (Show, Eq)

In [ ]:
insertBST :: Ord a => a -> BST a -> BST a
insertBST a Leaf = Node Leaf a Leaf
insertBST a tree@(Node lc a' rc)
    | a == a'   = tree
    | a > a'    = Node lc a' (insertBST a rc)
    | otherwise = Node (insertBST a lc) a' rc

In [ ]:
createBSTFromList :: Ord a => [a] -> BST a
createBSTFromList = foldr insertBST Leaf

In [ ]:
bst = createBSTFromList [4, 2, 3, 6, 7, 9, 1, 5, 8]

<h4>Tree Traversals</h4>

In [ ]:
inorder :: Show a => BST a -> String
inorder Leaf = ""
inorder (Node lc a rc) = inorder lc ++ show a ++ " " ++ inorder rc

In [ ]:
inorder bst

In [ ]:
preorder :: Show a => BST a -> String
preorder Leaf = ""
preorder (Node lc a rc) = show a ++ " " ++ preorder lc ++ preorder rc

In [ ]:
preorder bst

In [ ]:
postorder :: Show a => BST a -> String
postorder Leaf = ""
postorder (Node lc a rc) = postorder lc ++ postorder rc ++ " " ++ show a

In [ ]:
postorder bst

<h4>Heap</h4>

In [ ]:
type Rank = Int

data Heap a = EmptyHeap
            | HP a Rank (Heap a) (Heap a) deriving Show

In [ ]:
emptyHeap :: Ord a => Heap a
emptyHeap = EmptyHeap

In [ ]:
heapEmpty :: Ord a => Heap a -> Bool
heapEmpty EmptyHeap = True
heapEmpty _ = False

In [ ]:
findHeap :: Ord a => Heap a -> Maybe a
findHeap EmptyHeap = Nothing
findHeap (HP x _ _ _) = Just x

In [ ]:
rank :: Ord a => Heap a -> Int
rank EmptyHeap = 0
rank (HP _ r _ _) = r

In [ ]:
makeHP :: Ord a => a -> Heap a -> Heap a -> Heap a
makeHP x a b | rank a >= rank b = HP x (rank b + 1) a b
             | otherwise        = HP x (rank a + 1) b a
             
merge :: Ord a => Heap a -> Heap a -> Heap a
merge h EmptyHeap = h
merge EmptyHeap h = h
merge h1@(HP x _ a1 b1) h2@(HP y _ a2 b2)
    | x <= y    = makeHP x a1 (merge b1 h2)
    | otherwise = makeHP y a2 (merge h1 b2)

In [ ]:
insHeap :: Ord a => a -> Heap a -> Heap a
insHeap x h = merge (HP x 1 EmptyHeap EmptyHeap) h

In [ ]:
delHeap :: Ord a => Heap a -> Heap a
delHeap EmptyHeap = error "delHeap: empty heap"
delHeap (HP x _ a b) = merge a b

In [ ]:
newtype PQueue a = PQ (Heap a) deriving Show

emptyPQ = PQ emptyHeap

pqEmpty (PQ h) = heapEmpty h

enPQ v (PQ h) = PQ (insHeap v h)

frontPQ (PQ h) = findHeap h

delPQ (PQ h) = PQ (delHeap h)

In [ ]:
pq = enPQ 3 (enPQ 5 (enPQ 1 (enPQ 24 (enPQ 10 (enPQ 13 emptyPQ)))))

<h4>AVL Trees</h4>

In [1]:
data AVLTree a = EmptyAVL
               | NodeAVL a (AVLTree a) (AVLTree a) deriving Show

In [2]:
rotateLeft :: (Ord a, Show a) => AVLTree a -> AVLTree a
rotateLeft EmptyAVL = EmptyAVL
rotateLeft (NodeAVL v (NodeAVL lv lflf lfrt) rt) = NodeAVL lv lflf (NodeAVL v lfrt rt)

rotateRight :: (Ord a, Show a) => AVLTree a -> AVLTree a
rotateRight EmptyAVL = EmptyAVL
rotateRight (NodeAVL v lf (NodeAVL rv rtlf rtrt)) = NodeAVL rv (NodeAVL v lf rtlf) rtrt

In [3]:
dRotateLeftRight :: (Ord a, Show a) => AVLTree a -> AVLTree a
dRotateLeftRight (NodeAVL v (NodeAVL lfv lflf (NodeAVL lfrtv lfrtlf lfrtrt)) rt) =
    NodeAVL lfrtv (NodeAVL lfv lflf lfrtlf) (NodeAVL v lfrtrt rt)
    
dRotateRightLeft :: (Ord a, Show a) => AVLTree a -> AVLTree a
dRotateRightLeft (NodeAVL v lf (NodeAVL rtv (NodeAVL rtlfv rtlflf rtlfrt) rtrt)) =
    NodeAVL rtlfv (NodeAVL v lf rtlflf) (NodeAVL rtv rtlfrt rtrt)

In [4]:
height :: AVLTree a -> Int
height EmptyAVL = 0
height (NodeAVL _ left right) = 1 + max (height left) (height right)

In [5]:
addAVL :: (Ord a, Show a) => a -> AVLTree a -> AVLTree a
addAVL i EmptyAVL = NodeAVL i EmptyAVL EmptyAVL
addAVL i (NodeAVL v lf rt)
    | i < v       = let
                        newlf@(NodeAVL newlfv _ _) = addAVL i lf
                    in
                        if ((height newlf - height rt) == 2)
                        then if i < newlfv
                            then rotateLeft (NodeAVL v newlf rt)
                            else dRotateLeftRight (NodeAVL v newlf rt)
                        else (NodeAVL v newlf rt)
    | otherwise   = let
                        newrt@(NodeAVL newrtv _ _) = addAVL i rt
                    in
                        if ((height newrt - height lf) == 2)
                        then if i > newrtv
                            then rotateRight (NodeAVL v lf newrt)
                            else dRotateRightLeft (NodeAVL v lf newrt)
                        else (NodeAVL v lf newrt)

In [6]:
createAVLTreeFromList :: (Show a, Ord a) => [a] -> AVLTree a
createAVLTreeFromList = foldr addAVL EmptyAVL

In [7]:
tree = createAVLTreeFromList [6, 3, 7, 2, 4, 1]

In [12]:
tree''' = addAVL (-3) tree''

In [13]:
tree'''

NodeAVL 4 (NodeAVL 2 (NodeAVL (-1) (NodeAVL (-3) EmptyAVL EmptyAVL) (NodeAVL 1 EmptyAVL EmptyAVL)) (NodeAVL 3 EmptyAVL EmptyAVL)) (NodeAVL 6 (NodeAVL 5 EmptyAVL EmptyAVL) (NodeAVL 7 EmptyAVL EmptyAVL))